In [1]:
#install all of the packages
!pip config set global.trusted-host "pypi.org files.pythonhosted.org pypi.python.org"
!pip install python-dotenv >& /dev/null
!pip install openai >& /dev/null
!pip install --upgrade langchain >& /dev/null
!pip install pyyaml >& /dev/null
!pip install unstructured >& /dev/null
!pip install markdown >& /dev/null
!pip install chromadb >& /dev/null
!pip install tiktoken >& /dev/null

Writing to /Users/253617/.config/pip/pip.conf


In [2]:
#basic setup

import openai
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv, find_dotenv
import os

_ = load_dotenv(find_dotenv())
openai.api_key = os.environ["OPENAI_API_KEY"]

llm_model = 'gpt-3.5-turbo-0301'
llm = ChatOpenAI(temperature=0.8, model=llm_model)

In [3]:
import pprint
pp = pprint.PrettyPrinter(indent=4)

In [4]:
#
# Create all of the text strings that represent
#

import yaml

with open('./service_catalogue/services.yaml', 'r') as file:
    valid_services = yaml.safe_load(file)

pp.pprint(valid_services)
print("\n")

with open('./service_catalogue/service_descriptions.yaml', 'r') as file:
    descriptions = yaml.safe_load(file)

pp.pprint(descriptions)

[   'account',
    'authorization',
    'workload',
    'ingress',
    'transitGateway',
    'sqlDatabase',
    'objectStorage']


{   'account': 'Creates an AWS account that is secure and has entitlements for '
               'authorization',
    'authorization': 'creates groups and entitlements in active directoy that '
                     'can be used for authorization',
    'compute': 'Deploys containers together with everything required to '
               'support upgrades and reduce downtime',
    'ingress': 'Allows a deployed container to be routed to, deploys load '
               'balancer, DNS address, certificates, and supports both '
               'host-based routing and path-based routing',
    'objectStorage': 'creates a directory where objects can be stored.',
    'sqlDatabase': 'creates SQL databases',
    'transitGatway': 'Provisions a hub network that can route between '
                     'networks. Assigns ip addresses to avoid conflicts'}


In [5]:
# Create a prompt template that we can use to match queries to valid services from a service catalogue

from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

service_matcher_template = """ You are a helpful assistant that helps developers get their applications
running in the cloud.

For the following text, determine which service from the list of valid_services the developer is asking about.

valid_services: {valid_services}

text: {text}

The following json dictionary has each service as a key, and includes a high level
description as a value. {descriptions}

the output should be a list of valid servies being referenced

"""


service_matcher_prompt_template = PromptTemplate.from_template(service_matcher_template)

service_matcher_chain = LLMChain(
    llm = llm,
    prompt= service_matcher_prompt_template,
    verbose=False,
)


In [6]:
# the prompt template is aware of the inputs that can be compiled into the template
print(service_matcher_prompt_template.input_variables)

['descriptions', 'text', 'valid_services']


In [7]:
# let's run an example! here is a user question, let's see if our LLM can match what service it's referring to!
messages = "I want to route to my app using a dns address"

output = service_matcher_chain.predict(
    descriptions=descriptions,
    valid_services=valid_services,
    text=messages,
)
print(output)

The developer is asking about the 'ingress' service.


In [8]:
# Our LLM got the service right, but the format didn't return as we requested.

In [9]:
# We can pass the output back to the LLM and ask it again to return into a format that we
# can interact with programmatically

format_fixer_template_string = """ Take the text as input

Convert the text that describes a list of services into a list of service in json

text: the service is compute

["compute"]

text: {text}

"""

format_fixer_prompt_template = PromptTemplate.from_template(format_fixer_template_string) 

format_correcting_chain = LLMChain(
    llm = llm,
    prompt= format_fixer_prompt_template,
    verbose=False,
)

output = format_correcting_chain.predict(
    text=output,
)
print(output)

["ingress"]


In [10]:
# We not only want to understand which service the user is referring to, but also what their intent is.
# As mentioned before, we want to take the combination of intent + service so that our code can
# know what steps to perform

user_intent_template_str = """ You are a helpful assistant that helps developers get their applications
running in the cloud.

For the following text, determine if the user is asking a question, trying to provision a resource, or reporting an issue

the format of the response should be one of the following strings, "Asking a Question", "Reporting an issue", "Provisioning a resource"" 

text: {text}

"""

user_intent_template = PromptTemplate.from_template(user_intent_template_str) 

user_intent_llm_chain = LLMChain(
    llm = llm,
    prompt= user_intent_template,
    verbose=False,
)

In [11]:
#
# Let's see if our LLM can determine our intent here
#
message2 = "My database failed"
output = user_intent_llm_chain.predict(
    text=message2,
)
print(output)

Reporting an issue


In [12]:
#
# Let's try to figure out both the intent of a query and the service it relates to
#

message2 = "I want to deploy my container"

intent_output = user_intent_llm_chain.predict(
    text=message2,
)
service_type_output = service_matcher_chain.predict(
    descriptions=descriptions,
    valid_services=valid_services,
    text=messages,
)
print(service_type_output)
print(intent_output)

The service being referenced in the text is 'ingress'.
Provisioning a resource


In [13]:
# NOTE: it does not pick the right service, we can try to see if we can use embeddings to do a better job

In [14]:
#
# Split up our markdown documentation by splitting it up per section
#
from langchain.text_splitter import MarkdownHeaderTextSplitter

headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
]
markdown_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=headers_to_split_on
)

split_sections = []
for s in valid_services:
    with open(f'./service_catalogue/{s}.md', 'r') as file:
        read_file = file.read()
        md_header_splits = markdown_splitter.split_text(read_file)
        for i in md_header_splits:
            i.metadata['Document'] = f'{s}.md'
            print(i.metadata)
            split_sections.append(i)

{'Header 1': 'Overview', 'Document': 'account.md'}
{'Header 1': 'Inputs', 'Document': 'account.md'}
{'Header 1': 'Outputs', 'Document': 'account.md'}
{'Header 1': 'Overview', 'Document': 'authorization.md'}
{'Header 1': 'Overview', 'Document': 'workload.md'}
{'Header 1': 'Inputs', 'Document': 'workload.md'}
{'Header 1': 'outputs', 'Document': 'workload.md'}
{'Header 1': 'Overview', 'Document': 'ingress.md'}
{'Header 1': 'inputs', 'Document': 'ingress.md'}
{'Header 1': 'Overview', 'Document': 'transitGateway.md'}
{'Header 1': 'Inputs', 'Document': 'transitGateway.md'}
{'Header 1': 'Overview', 'Document': 'objectStorage.md'}
{'Header 1': 'inputs', 'Document': 'objectStorage.md'}


In [15]:
# We can get a vector representation
from langchain.embeddings.openai import OpenAIEmbeddings
embeddings_model = OpenAIEmbeddings()
punchlines = embeddings_model.embed_documents(
    "To get to the other side",
    "You forgot to say banana",
)
print(len(punchlines[0]))
print(punchlines[0][:4])
set_ups = embeddings_model.embed_query(
    "Why did the chicken cross the road?"
)
print(len(set_ups))
print(set_ups[:4])


1536
[-0.003914595988357722, -0.014495498282681086, 0.013440788521313102, -0.005354680506045646]
1536
[0.01918765961014604, -0.014656718335834696, -0.013342495483116794, 0.008755230611743351]


In [16]:
# Build a vectordb for our embeddings
from langchain.vectorstores import Chroma
persist_directory = 'docs/chroma/'

vectordb = Chroma.from_documents(
    persist_directory=persist_directory,
    documents=split_sections,
    embedding=embeddings_model,
)
print(vectordb._collection.count())

13


In [17]:
!mkdir -p docs/chroma

In [18]:
# Build a vectordb for our embeddings
from langchain.vectorstores import Chroma
persist_directory = 'docs/chroma/'

vectordb = Chroma.from_documents(
    persist_directory=persist_directory,
    documents=split_sections,
    embedding=embeddings_model,
)
print(vectordb._collection.count())

26


In [19]:
print(type(vectordb))
question = "How do I ensure there are no ip conflicts?"
docs = vectordb.similarity_search(question,k=5)

<class 'langchain.vectorstores.chroma.Chroma'>


In [20]:
pp.pprint(docs)

[   Document(page_content='The transit gateway services allows the provisioning of a network that\nallows you to route from the VPC in your cloud account to other VPCs.  \nIt does this both by configuring routes from your VPC to the transit\nnewtwork and also by configuring DNS in your account so that you can\nboth resolve DNS addresses across the network and so that you can\nleverage DNS addresses that can be resolved.  \nThe transit gateway also assigns you ip addresses using IPAM to ensure that\nyou do not get an overlapping IP.', metadata={'Document': 'transitGateway.md', 'Header 1': 'Overview'}),
    Document(page_content='The transit gateway services allows the provisioning of a network that\nallows you to route from the VPC in your cloud account to other VPCs.  \nIt does this both by configuring routes from your VPC to the transit\nnewtwork and also by configuring DNS in your account so that you can\nboth resolve DNS addresses across the network and so that you can\nleverage DNS

In [21]:
# Performing a similarity search on the embeddings from the markdown is far more effective for matching the service than
# above prompt.
# You can see that we have the most matches from our compute document, but also that the best match is from that document
question2 = "I want to deploy my container"
docs = vectordb.similarity_search(question2,k=5)

# which service from the service catalogue had the best match?

def get_best_matching_document(docs):
    return docs[0].metadata['Document']

print(get_best_matching_document(docs))

# which service catalogue document got the most matches?

def get_most_matching_document(docs):
    match_dict = {}
    for i in docs:
        doc_name = i.metadata['Document']
        if match_dict.get(doc_name):
            match_dict[doc_name] = match_dict[doc_name] + 1
        else:
            match_dict[doc_name] = 1 
    return match_dict


pp.pprint(get_most_matching_document(docs))

workload.md
{'transitGateway.md': 1, 'workload.md': 4}


In [22]:
#
# determine service and what the user wants, if they want to deploy a service, generate the spec
#

with open('./service_catalogue/compute.yaml', 'r') as file:
    compute_spec = yaml.safe_load(file)

def generate_spec(spec):
    spec_generate_string = """ You are a helpful assistant that helps developers generate the declarative specifications to get their resources running in the cloud.
        Generate a specicifcation that matches the following:
        {spec}
    """
    user_spec_template = PromptTemplate.from_template(spec_generate_string)
    user_spec_chain = LLMChain(
        llm = llm,
        prompt= user_spec_template,
        verbose=True,
    )
    return user_spec_chain.predict(spec=spec)

message = "I want to deploy my container"
intent = user_intent_llm_chain.predict(
    text=message,
)
docs = vectordb.similarity_search(message,k=5)
service = get_best_matching_document(docs)
print(intent)
print(service)

if intent == 'Provisioning a resource' and service == 'compute.md':
    output = generate_spec(compute_spec)
    print(output)

# it ignores my spec and generates a K8s spec!

FileNotFoundError: [Errno 2] No such file or directory: './service_catalogue/compute.yaml'

In [ ]:
from langchain.chains import RetrievalQA
llm_zero = ChatOpenAI(temperature=0.0, model=llm_model)
template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer. 
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)
qa_chain = RetrievalQA.from_chain_type(
    llm_zero,
    retriever=vectordb.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

result = qa_chain({"query": question})
print(result['result'])


In [ ]:
#
# if the user wants to report an issue, open a service now ticket on their behalf
#

message = "My SQL DB failed"
intent = user_intent_llm_chain.predict(
    text=message,
)
docs = vectordb.similarity_search(message,k=5)
service = get_best_matching_document(docs)
print(intent)
print(service)
print(get_most_matching_document(docs))


In [ ]:
# Why is is wrong?

with open('./service_catalogue/sqlDatabase.md', 'r') as file:
    print(file.read())

# I made up all of the docs, so I never filled the sqldatabase one in, so text, no embedding that are similar to the query

In [ ]:
message = "I can't access my buckets"
intent = user_intent_llm_chain.predict(
    text=message,
)
docs = vectordb.similarity_search(message,k=5)
service = get_best_matching_document(docs)
print(intent)
print(service)
print(get_most_matching_document(docs))

In [ ]:
from langchain.agents.agent_toolkits import create_python_agent
from langchain.tools.python.tool import PythonREPLTool
from langchain.python import PythonREPL
from langchain.llms.openai import OpenAI
from langchain.agents.agent_types import AgentType
from langchain.chat_models import ChatOpenAI
from langchain.agents import load_tools, initialize_agent
from langchain.agents import tool
from langchain.tools import StructuredTool

zero_llm = ChatOpenAI(temperature=0.0, model=llm_model)

def submit_service_now_ticket(service: str, query: str) -> str:
    """Calls out to service now to create a new ticket for the specific service passed in."""
    print(f'!!!!!submitting ticket to service now for {service}')

tool = StructuredTool.from_function(submit_service_now_ticket)

#agent = create_python_agent(
#    zero_llm,
#    tool=PythonREPLTool(),
#    verbose=True
#)

agent= initialize_agent(
    [tool], 
    zero_llm, 
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True
)

try:
    result = agent(f'Submit a ticket to service now with the following content {message} for the service {service}') 
    #print(results)
except: 
    print("exception on external access")